In [9]:
# Imports and display settings
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

pd.set_option("display.max_columns", None)
pd.set_option("display.width", 200)

In [10]:
# Load raw parquet files from the raw_data folder
raw = Path.cwd().parent

print("CWD:", Path.cwd())
print("raw exists:", raw.exists())

df_holidays = pd.read_parquet(raw / "20260218_144523_holidays.parquet", engine="pyarrow")
df_sales    = pd.read_parquet(raw / "20260218_144523_sales_data.parquet", engine="pyarrow")
df_stores   = pd.read_parquet(raw / "20260218_144523_stores.parquet", engine="pyarrow")
df_weather  = pd.read_parquet(raw / "20260218_144523_weather.parquet", engine="pyarrow")

print("holidays:", df_holidays.shape)
print("sales   :", df_sales.shape)
print("stores  :", df_stores.shape)
print("weather :", df_weather.shape)

CWD: c:\Users\simon\Food_Prediction\raw_data\code_raw_data_analysis
raw exists: True
holidays: (1301, 5)
sales   : (713637, 6)
stores  : (84, 5)
weather : (78624, 30)


In [11]:
# 1) Basic structure check for each table
for name, df in {
    "holidays": df_holidays,
    "sales": df_sales,
    "stores": df_stores,
    "weather": df_weather,
}.items():
    print(f"\n{name.upper()}")
    print("-" * 60)
    print("shape:", df.shape)
    print("columns:", list(df.columns))
    print("\ndtypes:")
    print(df.head(4))
    print(df.dtypes)


HOLIDAYS
------------------------------------------------------------
shape: (1301, 5)
columns: ['zipcode', 'subdivision_code', 'date', 'holiday_name', 'holiday_type']

dtypes:
  zipcode subdivision_code       date holiday_name    holiday_type
0   41812            DE-NW 2025-04-01   Zuckerfest     special_day
1   41812            DE-NW 2025-04-02   Zuckerfest     special_day
2   41812            DE-NW 2025-04-12  Osterferien  school_holiday
3   41812            DE-NW 2025-04-13  Osterferien  school_holiday
zipcode                        str
subdivision_code               str
date                datetime64[ns]
holiday_name                   str
holiday_type                   str
dtype: object

SALES
------------------------------------------------------------
shape: (713637, 6)
columns: ['date', 'category_name', 'item_id', 'sold_quantity', 'price', 'store_id']

dtypes:
        date         category_name  item_id  sold_quantity  price  store_id
0 2025-04-01     Angebot Brötchen       13

In [12]:
# 2) Missing values check (top 10 per table)
for name, df in {
    "holidays": df_holidays,
    "sales": df_sales,
    "stores": df_stores,
    "weather": df_weather,
}.items():
    print(f"\n{name.upper()} - missing values")
    print("-" * 60)
    missing = df.isna().sum().sort_values(ascending=False)
    print(missing[missing > 0].head(10) if (missing > 0).any() else "No missing values")


HOLIDAYS - missing values
------------------------------------------------------------
No missing values

SALES - missing values
------------------------------------------------------------
price    16989
dtype: int64

STORES - missing values
------------------------------------------------------------
No missing values

WEATHER - missing values
------------------------------------------------------------
No missing values


In [13]:
# 3) Key checks and uniqueness / duplicates

print("stores.store_id is unique:", df_stores["store_id"].is_unique)
print("stores.zipcode is unique:", df_stores["zipcode"].is_unique)

print("\nweather duplicates on (date, time, zipcode):",
      df_weather.duplicated(subset=["date", "time", "zipcode"]).sum())

print("weather duplicates on (date, zipcode):",
      df_weather.duplicated(subset=["date", "zipcode"]).sum())

print("\nholidays duplicates on (date, zipcode, holiday_name):",
      df_holidays.duplicated(subset=["date", "zipcode", "holiday_name"]).sum())

print("sales duplicates on (date, store_id, item_id):",
      df_sales.duplicated(subset=["date", "store_id", "item_id"]).sum())

stores.store_id is unique: True
stores.zipcode is unique: False

weather duplicates on (date, time, zipcode): 0
weather duplicates on (date, zipcode): 75348

holidays duplicates on (date, zipcode, holiday_name): 33
sales duplicates on (date, store_id, item_id): 2370


In [14]:
# 4) Granularity checks (rows per business key)

print("WEATHER rows per (date, zipcode):")
print(df_weather.groupby(["date", "zipcode"]).size().describe())

print("\nHOLIDAYS rows per (date, zipcode):")
print(df_holidays.groupby(["date", "zipcode"]).size().describe())

print("\nSALES rows per (date, store_id):")
print(df_sales.groupby(["date", "store_id"]).size().describe())

WEATHER rows per (date, zipcode):
count    3276.0
mean       24.0
std         0.0
min        24.0
25%        24.0
50%        24.0
75%        24.0
max        24.0
dtype: float64

HOLIDAYS rows per (date, zipcode):
count    1084.000000
mean        1.200185
std         0.441980
min         1.000000
25%         1.000000
50%         1.000000
75%         1.000000
max         4.000000
dtype: float64

SALES rows per (date, store_id):
count    7058.000000
mean      101.110371
std        62.414290
min         1.000000
25%        16.000000
50%       124.000000
75%       148.000000
max       227.000000
dtype: float64


In [15]:
# 5) Date coverage checks (min/max/unique dates)

for name, df in {
    "holidays": df_holidays,
    "sales": df_sales,
    "weather": df_weather,
}.items():
    print(f"\n{name.upper()} date range")
    print("min:", df["date"].min())
    print("max:", df["date"].max())
    print("unique dates:", df["date"].nunique())


HOLIDAYS date range
min: 2025-04-01 00:00:00
max: 2025-06-24 00:00:00
unique dates: 34

SALES date range
min: 2025-04-01 00:00:00
max: 2025-06-30 00:00:00
unique dates: 91

WEATHER date range
min: 2025-04-01 00:00:00
max: 2025-06-30 00:00:00
unique dates: 91


In [16]:
# 6) Coverage checks (store IDs and ZIP codes across tables)

sales_store_ids = set(df_sales["store_id"].unique())
store_master_ids = set(df_stores["store_id"].unique())

print("store_id in sales but missing in stores:", len(sales_store_ids - store_master_ids))
print("store_id in stores but not used in sales:", len(store_master_ids - sales_store_ids))

store_zips = set(df_stores["zipcode"].dropna().unique())
weather_zips = set(df_weather["zipcode"].dropna().unique())
holiday_zips = set(df_holidays["zipcode"].dropna().unique())

print("\nStore ZIPs missing in weather:", len(store_zips - weather_zips))
print("Store ZIPs missing in holidays:", len(store_zips - holiday_zips))

print("\nExample ZIPs missing in weather:", list(store_zips - weather_zips)[:10])
print("Example ZIPs missing in holidays:", list(store_zips - holiday_zips)[:10])

store_id in sales but missing in stores: 0
store_id in stores but not used in sales: 0

Store ZIPs missing in weather: 0
Store ZIPs missing in holidays: 0

Example ZIPs missing in weather: []
Example ZIPs missing in holidays: []


In [17]:
# 7) Target variable and core sales metrics checks

print("sold_quantity summary:")
print(df_sales["sold_quantity"].describe())

print("\nprice summary:")
print(df_sales["price"].describe())

print("\nNegative sold_quantity rows:", (df_sales["sold_quantity"] < 0).sum())
print("Negative price rows:", (df_sales["price"] < 0).sum())
print("Zero price rows:", (df_sales["price"] == 0).sum())

price_na = df_sales["price"].isna()
print("\nMissing price count:", price_na.sum())
print("Missing price share (%):", round(price_na.mean() * 100, 2))

print("\nTop categories with missing price:")
print(df_sales.loc[price_na, "category_name"].value_counts().head(10))

print("\nTop stores with missing price:")
print(df_sales.loc[price_na, "store_id"].value_counts().head(10))

sold_quantity summary:
count    713637.000000
mean         15.845915
std          60.843418
min          -1.000000
25%           1.000000
50%           4.000000
75%          12.000000
max        2425.000000
Name: sold_quantity, dtype: float64

price summary:
count    696648.000000
mean          2.747405
std           2.129056
min         -22.000000
25%           1.100000
50%           2.600000
75%           3.800000
max         115.000000
Name: price, dtype: float64

Negative sold_quantity rows: 2
Negative price rows: 2759
Zero price rows: 48261

Missing price count: 16989
Missing price share (%): 2.38

Top categories with missing price:
category_name
Brötchen            8799
Brot                8145
Snack                 33
Konditorei             8
Kaltgetränke 19%       4
Name: count, dtype: int64

Top stores with missing price:
store_id
6     1161
0     1140
4     1002
3      999
5      993
9      980
8      889
1      767
14     593
7      551
Name: count, dtype: int64


In [18]:
# 8) Weather plausibility checks (numeric ranges and key distributions)

weather_num_cols = [c for c in df_weather.columns if pd.api.types.is_numeric_dtype(df_weather[c])]

print("Weather numeric summary:")
print(df_weather[weather_num_cols].describe().T)

print("\nTop weather descriptions:")
print(df_weather["weather_description"].value_counts().head(15))

print("\nTop weather codes:")
print(df_weather["weather_code"].value_counts().head(15))

Weather numeric summary:
                    count         mean         std    min     25%     50%     75%     max
temperature       78624.0    13.819521    5.665801   -2.0    10.0    13.0    17.0    32.0
wind_speed        78624.0    13.142577    6.522310    0.0     9.0    12.0    17.0    39.0
wind_degree       78624.0   177.389474  101.179081    0.0    87.0   204.0   257.0   360.0
weather_code      78624.0   141.274267   57.894296  113.0   113.0   116.0   143.0   389.0
precip            78624.0     0.059655    0.288629    0.0     0.0     0.0     0.0     8.5
humidity          78624.0    69.556853   16.739121   23.0    57.0    71.0    84.0    99.0
visibility        78624.0     9.577101    1.750297    0.0    10.0    10.0    10.0    10.0
pressure          78624.0  1018.022563    5.612091  998.0  1015.0  1018.0  1022.0  1029.0
cloudcover        78624.0    39.178139   35.601236    0.0     7.0    26.0    74.0   100.0
heatindex         78624.0    13.974295    5.907601   -2.0    10.0    13.0  

In [19]:
# 9) Holiday plausibility checks

print("Holiday type distribution:")
print(df_holidays["holiday_type"].value_counts())

print("\nTop holiday names:")
print(df_holidays["holiday_name"].value_counts().head(20))

print("\nSubdivision code consistency check (stores vs holidays)")
store_subdiv = set(df_stores["subdivision_code"].dropna().unique())
holiday_subdiv = set(df_holidays["subdivision_code"].dropna().unique())

print("Subdivision codes in stores but not in holidays:", len(store_subdiv - holiday_subdiv))
print("Subdivision codes in holidays but not in stores:", len(holiday_subdiv - store_subdiv))

print("\nExamples (stores not in holidays):", list(store_subdiv - holiday_subdiv)[:10])
print("Examples (holidays not in stores):", list(holiday_subdiv - store_subdiv)[:10])

Holiday type distribution:
holiday_type
school_holiday    635
special_day       443
holiday           223
Name: count, dtype: int64

Top holiday names:
holiday_name
Osterferien                  598
Brückentag                   110
Zuckerfest                    74
Palmsonntag                   37
Gründonnerstag                37
Karfreitag                    37
Karsamstag                    37
Ostermontag                   37
Walpurgisnacht                37
Tag der Arbeit                37
Muttertag                     37
Christi Himmelfahrt           37
Internationaler Kindertag     37
Pfingstmontag                 37
Pfingstferien                 37
Johannistag                   37
Fronleichnam                  36
Ostersonntag                   1
Pfingstsonntag                 1
Name: count, dtype: int64

Subdivision code consistency check (stores vs holidays)
Subdivision codes in stores but not in holidays: 0
Subdivision codes in holidays but not in stores: 1

Examples (stores not i

In [20]:
# Optional environment check
import sys, pyarrow, pandas
print(sys.executable)
print("pyarrow:", pyarrow.__version__)
print("pandas :", pandas.__version__)

c:\Users\simon\Food_Prediction\.venv\Scripts\python.exe
pyarrow: 23.0.1
pandas : 3.0.1
